In [54]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel
rec_system = pd.read_parquet("../datasets/rec_system.parquet").head(3000)
rec_system_copy = rec_system.copy()

rec_system_copy.fillna({"overview":"[Unknown]",
                   "name_genre":"[Unknown]",
                   "actors_names":"[Unknown]",
                   "director_names":"[Unknown]",
                   "tagline":"[Unknown]",
                   "company":"[Unknown]",
                   },inplace=True)


# rec_system_copy["overview"] = rec_system_copy["overview"].apply(lambda x: x.split())
# rec_system_copy["tagline"] = rec_system_copy["tagline"].apply(lambda x: x.split())






    

In [55]:
rec_system_copy

,title,overview,name_genre,actors_names,director_names,tagline,company,collection,release_year,vote_average,popularity
0,toy story,"Led by Woody, Andy's toys live happily in his ...","[Animation, Comedy, Family]","[Laurie Metcalf, Sarah Freeman, Erik von Dette...",[John Lasseter],[Unknown],Pixar Animation Studios,Toy Story Collection,1995,7.7,21.946943
1,jumanji,When siblings Judy and Peter discover an encha...,"[Adventure, Fantasy, Family]","[Sarah Gilson, Leonard Zola, Peter Bryant, Lau...",[Joe Johnston],Roll the dice and unleash the excitement!,TriStar Pictures Teitler Film Interscope Commu...,[],1995,6.9,17.015539
2,grumpier old men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]","[Walter Matthau, Kevin Pollak, Jack Lemmon, Bu...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,Warner Bros. Lancaster Gate,Grumpy Old Men Collection,1995,6.5,11.712900
3,waiting to exhale,"Cheated on, mistreated and stepped on, the wom...","[Romance, Drama, Comedy]","[Michael Beach, Lamont Johnson, Mykelti Willia...",[Forest Whitaker],Friends are the people who let you be yourself...,Twentieth Century Fox Film Corporation,[],1995,6.1,3.859495
4,father of the bride part ii,Just when George Banks has recovered from his ...,[Comedy],"[Martin Short, BD Wong, George Newbern, Steve ...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,Sandollar Productions Touchstone Pictures,Father of the Bride Collection,1995,5.7,8.387519
...,...,...,...,...,...,...,...,...,...,...,...
2995,cradle will rock,A true story of politics and art in the 1930s ...,[Drama],"[Cherry Jones, Rubén Blades, Emily Watson, Jam...",[Tim Robbins],[Unknown],Touchstone Pictures,[],1999,7.2,6.481649
2996,deuce bigalow male gigolo,"Deuce Bigalow is a less than attractive, down ...",[Comedy],"[Bree Turner, Deborah Lemen, Jackie Sandler, T...",[Mike Mitchell],He charges $10 but he’s willing to negotiate.,Happy Madison Productions Quinta Communication...,Deuce Bigalow Collection,1999,5.4,6.567794
2997,the green mile,A supernatural tale set on death row in a Sout...,"[Crime, Drama, Fantasy]","[Mack Miles, Michael Clarke Duncan, Brent Bris...",[Frank Darabont],Miracles do happen.,Castle Rock Entertainment Darkwoods Production...,[],1999,8.2,19.966780
2998,the cider house rules,"Homer is an orphan who was never adopted, beco...",[Drama],"[Paul Rudd, Tobey Maguire, Erik Per Sullivan, ...",[Lasse Hallström],A story about how far we must travel to find t...,Miramax Films Nina Saxon Film Design FilmColony,[],1999,6.9,9.526781


In [56]:
def collapse(valor):
    valores =[]
    for i in valor:
       valores.append(i)
    return valores


# rec_system_copy["company"]=rec_system_copy["company"].apply(collapse)
rec_system_copy["name_genre"]=rec_system_copy["name_genre"].apply(collapse)
# rec_system_copy["actors_names"]=rec_system_copy["actors_names"].apply(collapse)
# rec_system_copy["director_names"]=rec_system_copy["director_names"].apply(collapse)

rec_system_copy["name_genre"] = rec_system_copy["name_genre"].apply(lambda x: ",".join(x))
# rec_system_copy["company"] = rec_system_copy["company"].apply(lambda x: ",".join(x))

rec_system_copy["tags"] =  rec_system_copy["name_genre"]  + " " + rec_system_copy["title"] + str(rec_system_copy["vote_average"]) + " "       #+ rec_system_copy["actors_names"] + rec_system_copy["director_names"]  

rec_system_copy["tags"] = rec_system_copy["tags"].apply(lambda x: "".join(x.replace(","," ")))

In [57]:
rec_system_copy

,title,overview,name_genre,actors_names,director_names,tagline,company,collection,release_year,vote_average,popularity,tags
0,toy story,"Led by Woody, Andy's toys live happily in his ...","Animation,Comedy,Family","[Laurie Metcalf, Sarah Freeman, Erik von Dette...",[John Lasseter],[Unknown],Pixar Animation Studios,Toy Story Collection,1995,7.7,21.946943,Animation Comedy Family toy story0 7.7\n...
1,jumanji,When siblings Judy and Peter discover an encha...,"Adventure,Fantasy,Family","[Sarah Gilson, Leonard Zola, Peter Bryant, Lau...",[Joe Johnston],Roll the dice and unleash the excitement!,TriStar Pictures Teitler Film Interscope Commu...,[],1995,6.9,17.015539,Adventure Fantasy Family jumanji0 7.7\n1...
2,grumpier old men,A family wedding reignites the ancient feud be...,"Romance,Comedy","[Walter Matthau, Kevin Pollak, Jack Lemmon, Bu...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,Warner Bros. Lancaster Gate,Grumpy Old Men Collection,1995,6.5,11.712900,Romance Comedy grumpier old men0 7.7\n1 ...
3,waiting to exhale,"Cheated on, mistreated and stepped on, the wom...","Romance,Drama,Comedy","[Michael Beach, Lamont Johnson, Mykelti Willia...",[Forest Whitaker],Friends are the people who let you be yourself...,Twentieth Century Fox Film Corporation,[],1995,6.1,3.859495,Romance Drama Comedy waiting to exhale0 ...
4,father of the bride part ii,Just when George Banks has recovered from his ...,Comedy,"[Martin Short, BD Wong, George Newbern, Steve ...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,Sandollar Productions Touchstone Pictures,Father of the Bride Collection,1995,5.7,8.387519,Comedy father of the bride part ii0 7.7\...
...,...,...,...,...,...,...,...,...,...,...,...,...
2995,cradle will rock,A true story of politics and art in the 1930s ...,Drama,"[Cherry Jones, Rubén Blades, Emily Watson, Jam...",[Tim Robbins],[Unknown],Touchstone Pictures,[],1999,7.2,6.481649,Drama cradle will rock0 7.7\n1 6.9...
2996,deuce bigalow male gigolo,"Deuce Bigalow is a less than attractive, down ...",Comedy,"[Bree Turner, Deborah Lemen, Jackie Sandler, T...",[Mike Mitchell],He charges $10 but he’s willing to negotiate.,Happy Madison Productions Quinta Communication...,Deuce Bigalow Collection,1999,5.4,6.567794,Comedy deuce bigalow male gigolo0 7.7\n1...
2997,the green mile,A supernatural tale set on death row in a Sout...,"Crime,Drama,Fantasy","[Mack Miles, Michael Clarke Duncan, Brent Bris...",[Frank Darabont],Miracles do happen.,Castle Rock Entertainment Darkwoods Production...,[],1999,8.2,19.966780,Crime Drama Fantasy the green mile0 7.7\...
2998,the cider house rules,"Homer is an orphan who was never adopted, beco...",Drama,"[Paul Rudd, Tobey Maguire, Erik Per Sullivan, ...",[Lasse Hallström],A story about how far we must travel to find t...,Miramax Films Nina Saxon Film Design FilmColony,[],1999,6.9,9.526781,Drama the cider house rules0 7.7\n1 ...


In [58]:
tv = TfidfVectorizer(max_features=5000, stop_words="english")
vector = tv.fit_transform(rec_system_copy["tags"]).toarray()

cosine_sim = linear_kernel(vector,vector)

indices = pd.Series(rec_system_copy.index, index=rec_system_copy["title"]).drop_duplicates()

def recomendacion(movie:str):
    movie = movie.lower()
    num=5
    try:
        index = indices[movie]
        sim_scores=list(enumerate(cosine_sim[index]))
        distances = sorted(sim_scores,reverse= True,key=lambda x: x[1])
        lista=[]
        for i in distances[1:num+1]:
            # print(i[0])
            titulo = rec_system_copy.iloc[i[0]].title
            lista.append(titulo.title())
        return lista
    except: 
        return f"La pelicula {movie} no se encuentra en la base de datos"
    
recomendacion("mission impossible")

['The Rock', 'Goldeneye', 'Cliffhanger', 'Daylight', 'Firestorm']